In [ ]:
# Install the required dependencies
!pip install langchain==0.0.189 openai tiktoken pypdf

In [2]:
from langchain.document_loaders import PyPDFLoader

# Load the book
loader = PyPDFLoader("crime-and-punishment.pdf")
pages = loader.load()
pages
# Cut out the open and closing parts
pages = pages[6:743]

# Combine the pages, and replace the tabs with spaces
text = ""

for page in pages:
    text += page.page_content

text = text.replace('\t', ' ')

In [4]:
text[0:2000]

'\x18 Free eBooks at Planet eBook.comChapter I\nOn an exceptionally hot evening early in July a young \nman came out of the garret in which he lodged in S. \nPlace and walked slowly, as though in hesitation, towards \nK. bridge.\nHe had successfully avoided meeting his landlady on the \nstaircase. His garret was under the roof of a high, five-sto -\nried house and was more like a cupboard than a room. The \nlandlady who provided him with garret, dinners, and at -\ntendance, lived on the floor below, and every time he went \nout he was obliged to pass her kitchen, the door of which \ninvariably stood open. And each time he passed, the young \nman had a sick, frightened feeling, which made him scowl \nand feel ashamed. He was hopelessly in debt to his landlady, \nand was afraid of meeting her.\nThis was not because he was cowardly and abject, quite \nthe contrary; but for some time past he had been in an over -\nstrained irritable condition, verging on hypochondria. He \nhad become so co

In [7]:
import re


cleaned_text = re.sub(r'\s*Free eBooks at Planet eBook\.com\s*', '', text, flags=re.DOTALL)
cleaned_text = re.sub(r' +', ' ', cleaned_text)           # Remove extra spaces
cleaned_text = re.sub(r'(Crime and Punishment )?[\x00-\x1F]', '', cleaned_text)  # Remove non-printable characters
cleaned_text = cleaned_text.replace('\n', ' ')            # Replace newline characters with spaces
cleaned_text = re.sub(r'\s*-\s*', '', cleaned_text)

# Displaying the first 1000 characters after initial cleanup for review
cleaned_text[:2000]


'Chapter IOn an exceptionally hot evening early in July a young man came out of the garret in which he lodged in S. Place and walked slowly, as though in hesitation, towards K. bridge.He had successfully avoided meeting his landlady on the staircase. His garret was under the roof of a high, fivestoried house and was more like a cupboard than a room. The landlady who provided him with garret, dinners, and attendance, lived on the floor below, and every time he went out he was obliged to pass her kitchen, the door of which invariably stood open. And each time he passed, the young man had a sick, frightened feeling, which made him scowl and feel ashamed. He was hopelessly in debt to his landlady, and was afraid of meeting her.This was not because he was cowardly and abject, quite the contrary; but for some time past he had been in an overstrained irritable condition, verging on hypochondria. He had become so completely absorbed in himself, and isolated from his fellows that he dreaded mee

'Chapter IOn an exceptionally hot evening early in July a young man came out of the garret in which he lodged in S. Place and walked slowly, as though in hesitation, towards K. bridge.He had successfully avoided meeting his landlady on the staircase. His garret was under the roof of a high, fivestoried house and was more like a cupboard than a room. The landlady who provided him with garret, dinners, and attendance, lived on the floor below, and every time he went out he was obliged to pass her kitchen, the door of which invariably stood open. And each time he passed, the young man had a sick, frightened feeling, which made him scowl and feel ashamed. He was hopelessly in debt to his landlady, and was afraid of meeting her.This was not because he was cowardly and abject, quite the contrary; but for some time past he had been in an overstrained irritable condition, verging on hypochondria. He had become so completely absorbed in himself, and isolated from his fellows that he dreaded mee